In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import librosa
import librosa.display
from IPython.display import Audio
import warnings
warnings.filterwarnings('ignore')

In [ ]:
list_emotion = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised"
}

In [ ]:
paths = []
labels = []
for dirname, _, filenames in os.walk('drive/MyDrive/mini_project/dataset'):
    for filename in filenames:
        paths.append(os.path.join(dirname, filename))
        basename = os.path.basename(filename)  # get the base name of the audio file

        emotion = list_emotion[basename.split("-")[2]]
        labels.append(emotion)


print('Dataset is Loaded')

Dataset is Loaded


In [ ]:
len(paths)

2900

In [ ]:
paths[:5]

['drive/MyDrive/mini_project/dataset/Actor_16/03-01-03-01-01-01-16.wav',
 'drive/MyDrive/mini_project/dataset/Actor_16/03-01-06-01-02-01-16.wav',
 'drive/MyDrive/mini_project/dataset/Actor_16/03-01-01-01-02-01-16.wav',
 'drive/MyDrive/mini_project/dataset/Actor_16/03-01-02-02-01-02-16.wav',
 'drive/MyDrive/mini_project/dataset/Actor_16/03-01-05-02-01-02-16.wav']

In [ ]:
labels[:5]

['happy', 'fearful', 'neutral', 'calm', 'angry']

In [ ]:
df = pd.DataFrame()
df['speech'] = paths
df['label'] = labels
df.head()

,speech,label
0,drive/MyDrive/mini_project/dataset/Actor_16/03...,happy
1,drive/MyDrive/mini_project/dataset/Actor_16/03...,fearful
2,drive/MyDrive/mini_project/dataset/Actor_16/03...,neutral
3,drive/MyDrive/mini_project/dataset/Actor_16/03...,calm
4,drive/MyDrive/mini_project/dataset/Actor_16/03...,angry


In [ ]:
df['label'].value_counts()

calm         398
happy        390
fearful      384
angry        384
sad          384
disgust      384
surprised    384
neutral      192
Name: label, dtype: int64

In [ ]:
def extract_feature(filename):
    data, sample_rate = librosa.load(filename)
    stft = np.abs(librosa.stft(data))
    mfccs = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate, n_mfcc=40).T, axis=0)
    mel = np.mean(librosa.feature.melspectrogram(data, sr=sample_rate).T,axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    return mfccs

In [ ]:
extract_feature(df['speech'][0])

array([-5.9386768e+02,  4.2766674e+01, -1.3588821e+01,  9.2966251e+00,
       -4.3494596e+00, -7.3283110e+00, -1.1642889e+01, -8.2818937e+00,
       -7.3986855e+00,  1.1769481e+00, -1.1669778e+01, -2.4123898e+00,
       -1.8806301e+00, -4.5035563e+00, -3.9620538e+00, -5.9451132e+00,
       -6.2600498e+00, -2.4723210e+00, -6.0073075e+00, -2.2582023e+00,
       -4.9507222e+00, -4.5176630e+00, -4.7073860e+00, -1.5303017e+00,
       -4.2095904e+00,  2.6708980e+00,  4.8840713e-02,  2.0904801e+00,
       -1.9206530e+00, -2.0623198e+00, -3.4813678e+00,  4.3015122e-02,
        6.5309185e-01,  4.5408354e+00,  2.8296535e+00,  5.8420634e+00,
        2.7520027e+00,  1.3423792e+00, -9.6067172e-01,  6.5380953e-02],
      dtype=float32)

In [ ]:
!nvidia-smi

Tue Dec  6 06:19:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P0    29W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
x_mfccs = df['speech'].apply(lambda x: extract_feature(x))

In [ ]:
x_mfccs

0       [-593.8677, 42.766674, -13.588821, 9.296625, -...
1       [-584.0855, 39.733234, -2.819869, 10.983516, -...
2       [-634.3659, 53.52496, 1.9311182, 10.183448, -1...
3       [-617.10803, 57.961075, -1.6098831, 11.663807,...
4       [-426.69778, 30.02772, -20.288496, 4.213167, -...
                              ...                        
2895    [-537.498, 33.912415, -17.367428, 1.641448, -1...
2896    [-576.32227, 45.961117, -18.4776, 6.270887, -1...
2897    [-618.717, 52.87845, -10.296721, 5.4750185, -6...
2898    [-550.753, 48.40233, -16.642029, 0.59691024, -...
2899    [-559.0442, 47.58398, -17.176203, 2.4187784, -...
Name: speech, Length: 2900, dtype: object

In [ ]:
def extract_feature_4(filename):
    data, sample_rate = librosa.load(filename)
    stft = np.abs(librosa.stft(data))

    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    return contrast

In [ ]:
x_contrast = df['speech'].apply(lambda x: extract_feature_4(x))

In [ ]:
x_contrast

0       [19.435276903820533, 16.64579667440855, 18.036...
1       [16.109572509579657, 15.35540299508464, 16.756...
2       [18.93725247510756, 15.633540928174398, 17.859...
3       [17.77508695327363, 15.18141954088422, 16.6121...
4       [14.869500633884035, 16.104168992151173, 17.33...
                              ...                        
2895    [12.401401521991385, 13.462099429514796, 15.68...
2896    [19.257106109444656, 16.517123641532084, 16.69...
2897    [20.016517107752957, 16.348092385952583, 17.41...
2898    [18.75775986664647, 15.981829134527645, 16.712...
2899    [18.86399325383258, 15.861253116469742, 16.013...
Name: speech, Length: 2900, dtype: object

In [ ]:
x_contrast[0]

array([19.4352769 , 16.64579667, 18.03651271, 17.10428527, 17.93069727,
       18.53633056, 42.16982881])

In [ ]:
def extract_feature_3(filename):
    data, sample_rate = librosa.load(filename)
    stft = np.abs(librosa.stft(data))

    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    return chroma

In [ ]:
x_chroma = df['speech'].apply(lambda x: extract_feature_3(x))

In [ ]:
len(x_chroma[0])

12

In [ ]:
x_mfccs = list(x_mfccs)
x_contrast = list(x_contrast)
x_chroma = list(x_chroma)
from itertools import chain
x_da = [list(chain.from_iterable(x)) for x in zip(x_mfccs,x_contrast,x_chroma)]
len(x_da[0])

59

In [ ]:
 da = pd.DataFrame(x_da, columns =['m1','m2','m3','m4','m5','m6','m7','m8','m9','m10','m11','m12','m13','m14','m15','m16','m17','m18','m19','m20','m21','m22','m23','m24','m25','m26','m27','m28','m29','m30','m31','m32','m33','m34','m35','m36','m37','m38','m39','m40','c1','c2','c3','c4','c5','c6','c7','h1','h2','h3','h4','h5','h6','h7','h8','h9','h10','h11','h12'])
 da

,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,...,h3,h4,h5,h6,h7,h8,h9,h10,h11,h12
0,-593.867676,42.766674,-13.588821,9.296625,-4.349460,-7.328311,-11.642889,-8.281894,-7.398685,1.176948,...,0.534104,0.490328,0.418054,0.412552,0.516447,0.567496,0.528193,0.520183,0.531675,0.561040
1,-584.085510,39.733234,-2.819869,10.983516,-1.897791,-6.922876,-14.886739,-9.417712,-5.231410,1.352311,...,0.595505,0.574596,0.508687,0.490343,0.550999,0.560133,0.625444,0.707241,0.715302,0.778056
2,-634.365906,53.524960,1.931118,10.183448,-1.134427,-9.264474,-16.133320,-5.179246,-7.692840,1.537901,...,0.651007,0.600976,0.605563,0.618894,0.610335,0.649571,0.664244,0.659528,0.667636,0.690084
3,-617.108032,57.961075,-1.609883,11.663807,2.932618,-6.709863,-11.051159,-4.720947,-4.965144,0.344765,...,0.586239,0.577910,0.588816,0.593688,0.607869,0.610051,0.651315,0.742181,0.711857,0.643531
4,-426.697784,30.027719,-20.288496,4.213167,-12.470323,-10.929867,-6.361925,-12.002077,-13.903050,5.385250,...,0.557655,0.544142,0.530530,0.558295,0.586619,0.541569,0.536285,0.568345,0.551319,0.571638
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2895,-537.497986,33.912415,-17.367428,1.641448,-13.561514,-10.013611,-6.788619,-9.947794,-9.232779,-3.985822,...,0.691943,0.684802,0.608561,0.577731,0.619962,0.669661,0.680618,0.696035,0.723051,0.745801
2896,-576.322266,45.961117,-18.477600,6.270887,-15.408111,-2.556136,-9.277953,-12.990032,-7.808734,0.290190,...,0.479396,0.495040,0.493665,0.513156,0.542380,0.561550,0.599118,0.572351,0.573544,0.575123
2897,-618.716980,52.878448,-10.296721,5.475019,-6.781945,-1.335837,-9.393105,-11.687968,-7.041688,1.007899,...,0.434423,0.448702,0.455442,0.533610,0.546764,0.531184,0.527218,0.577220,0.587715,0.538401
2898,-550.752991,48.402328,-16.642029,0.596910,-12.640155,-3.948209,-14.291334,-14.503586,-6.559266,-1.528507,...,0.535383,0.540516,0.505517,0.521778,0.565526,0.610539,0.592268,0.614464,0.652888,0.683912


In [ ]:
def waveplot(data, sr, emotion):
    plt.figure(figsize=(10,4))
    plt.title(emotion, size=20)
    librosa.display.waveplot(data, sr=sr)
    plt.show()

def spectogram(data, sr, emotion):
    x = librosa.stft(data)
    xdb = librosa.amplitude_to_db(abs(x))
    plt.figure(figsize=(11,4))
    plt.title(emotion, size=20)
    librosa.display.specshow(xdb, sr=sr, x_axis='time', y_axis='hz')
    plt.colorbar()

In [ ]:
def extract_feature_5(filename):
    data, sample_rate = librosa.load(filename)
    return data

In [ ]:
def extract_feature_6(filename):
    data, sample_rate = librosa.load(filename)
    return sample_rate

In [ ]:
X = [x for x in x_da]
X = np.array(X)
X.shape

(2900, 59)

In [ ]:
## input split
X = np.expand_dims(X, -1)
X.shape

(2900, 59, 1)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
y = enc.fit_transform(df[['label']])

In [ ]:
y = y.toarray()

In [ ]:
y.shape

(2900, 8)

Create the LSTM **Model:**

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 44)

# determining the shapes of training and testing sets
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(2175, 59, 1)
(2175, 8)
(725, 59, 1)
(725, 8)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

model = Sequential([
    LSTM(256, return_sequences=False, input_shape=(59,1)),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(8, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 256)               264192    
                                                                 
 dropout_6 (Dropout)         (None, 256)               0         
                                                                 
 dense_6 (Dense)             (None, 128)               32896     
                                                                 
 dropout_7 (Dropout)         (None, 128)               0         
                                                                 
 dense_7 (Dense)             (None, 64)                8256      
                                                                 
 dropout_8 (Dropout)         (None, 64)                0         
                                                                 
 dense_8 (Dense)             (None, 8)                

In [ ]:
history = model.fit(x_train, y_train, validation_split=0.2, epochs=100, batch_size=64)

Epoch 1/100
28/28 [==============================] - 10s 267ms/step - loss: 2.0791 - accuracy: 0.1316 - val_loss: 2.0716 - val_accuracy: 0.1218
Epoch 2/100
28/28 [==============================] - 7s 245ms/step - loss: 2.0636 - accuracy: 0.1425 - val_loss: 2.0748 - val_accuracy: 0.1264
Epoch 3/100
28/28 [==============================] - 7s 243ms/step - loss: 2.0656 - accuracy: 0.1276 - val_loss: 2.0689 - val_accuracy: 0.1218
Epoch 4/100
28/28 [==============================] - 7s 244ms/step - loss: 2.0645 - accuracy: 0.1397 - val_loss: 2.0724 - val_accuracy: 0.1540
Epoch 5/100
28/28 [==============================] - 7s 246ms/step - loss: 2.0615 - accuracy: 0.1385 - val_loss: 2.0679 - val_accuracy: 0.1402
Epoch 6/100
28/28 [==============================] - 7s 246ms/step - loss: 2.0457 - accuracy: 0.1609 - val_loss: 2.0720 - val_accuracy: 0.1724
Epoch 7/100
28/28 [==============================] - 7s 246ms/step - loss: 2.0178 - accuracy: 0.1994 - val_loss: 1.9534 - val_accuracy: 0.243